In [1]:
# UNINSTALL PyTorch versi CPU terlebih dahulu
!pip uninstall torch torchvision torchaudio -y

# INSTALL PyTorch versi CUDA 11.8
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.7.1+cu118
Uninstalling torch-2.7.1+cu118:
  Successfully uninstalled torch-2.7.1+cu118
Found existing installation: torchvision 0.22.1+cu118
Uninstalling torchvision-0.22.1+cu118:
  Successfully uninstalled torchvision-0.22.1+cu118
Found existing installation: torchaudio 2.7.1+cu118
Uninstalling torchaudio-2.7.1+cu118:
  Successfully uninstalled torchaudio-2.7.1+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp312-cp312-win_amd64.whl (2817.2 MB)
Using cached https://download.pytorch.org/whl/cu118/torchv

In [2]:
!pip install torchmetrics pyqubo dwave-neal

In [1]:
# Cek instalasi CUDA dan PyTorch secara detail
import torch
import sys

print("=" * 60)
print("DIAGNOSIS LENGKAP GPU/CUDA")
print("=" * 60)

# 1. Cek versi Python
print(f"\n1. Python Version: {sys.version}")

# 2. Cek versi PyTorch
print(f"\n2. PyTorch Version: {torch.__version__}")

# 3. Cek apakah PyTorch di-compile dengan CUDA
print(f"\n3. PyTorch compiled with CUDA: {torch.cuda.is_available()}")
print(f"   CUDA built version: {torch.version.cuda if torch.version.cuda else 'TIDAK ADA - PyTorch versi CPU!'}")

# 4. Cek jumlah GPU
print(f"\n4. Jumlah GPU terdeteksi: {torch.cuda.device_count()}")

# 5. Jika CUDA tersedia, tampilkan info detail
if torch.cuda.is_available():
    print(f"\n5. ✓ GPU TERDETEKSI!")
    print(f"   - Nama GPU: {torch.cuda.get_device_name(0)}")
    print(f"   - CUDA Capability: {torch.cuda.get_device_capability(0)}")
    print(f"   - Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"   - Current Device: {torch.cuda.current_device()}")
else:
    print(f"\n5. ✗ GPU TIDAK TERDETEKSI!")
    print(f"\n   KEMUNGKINAN PENYEBAB:")
    print(f"   ❌ Driver NVIDIA belum terinstall atau versi lama")
    print(f"   ❌ CUDA Toolkit tidak terinstall")
    print(f"   ❌ PyTorch terinstall versi CPU (bukan CUDA)")
    print(f"   ❌ GPU tidak kompatibel dengan versi CUDA")
    
    print(f"\n   SOLUSI:")
    print(f"   1. Cek GPU Anda: Apakah NVIDIA GPU?")
    print(f"   2. Install/Update Driver NVIDIA terbaru dari:")
    print(f"      https://www.nvidia.com/Download/index.aspx")
    print(f"   3. Setelah install driver, restart komputer")
    print(f"   4. Jalankan sel ini lagi untuk verifikasi")

print("\n" + "=" * 60)

DIAGNOSIS LENGKAP GPU/CUDA

1. Python Version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]

2. PyTorch Version: 2.7.1+cu118

3. PyTorch compiled with CUDA: True
   CUDA built version: 11.8

4. Jumlah GPU terdeteksi: 1

5. ✓ GPU TERDETEKSI!
   - Nama GPU: NVIDIA GeForce RTX 3080
   - CUDA Capability: (8, 6)
   - Total Memory: 10.00 GB
   - Current Device: 0



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
from tqdm.notebook import tqdm
import os
import cv2
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

from pyqubo import Binary, Array, Constraint, solve_qubo
from neal import SimulatedAnnealingSampler

# Check CUDA availability and set device
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✓ CUDA tersedia! Menggunakan GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ Jumlah GPU: {torch.cuda.device_count()}")
    print(f"✓ CUDA Version: {torch.version.cuda}")
    print(f"✓ Memory GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    device = torch.device('cpu')
    print("⚠ CUDA tidak tersedia. Menggunakan CPU.")
    print("⚠ Untuk menggunakan GPU, install PyTorch dengan CUDA:")
    print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

# Initialize metrics from torchmetrics
ssim = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
psnr = PeakSignalNoiseRatio(data_range=1.0).to(device)

# Define evaluation function
def evaluate_model(model, dataloader, device=device):
    """
    Evaluate model with PSNR and SSIM metrics
    
    Args:
        model: Neural network model
        dataloader: DataLoader for evaluation data
        device: Device to run evaluation on (cuda/cpu)
    
    Returns:
        avg_loss, avg_psnr, avg_ssim: Average metrics across dataset
    """
    model.to(device)
    model.eval()
    total_loss = 0.0
    total_psnr = 0.0
    total_ssim = 0.0
    loss_func = torch.nn.L1Loss()

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss_val = loss_func(outputs, targets)
            psnr_val = psnr(outputs, targets)
            ssim_val = ssim(outputs, targets)
            total_loss += loss_val.item()
            total_psnr += psnr_val.item()
            total_ssim += ssim_val.item()

    avg_loss = total_loss / len(dataloader)
    avg_psnr = total_psnr / len(dataloader)
    avg_ssim = total_ssim / len(dataloader)

    return avg_loss, avg_psnr, avg_ssim

print(f"\n✓ Device aktif: {device}")
print("✓ Semua fungsi dan metrics siap digunakan!")

✓ CUDA tersedia! Menggunakan GPU: NVIDIA GeForce RTX 3080
✓ Jumlah GPU: 1
✓ CUDA Version: 11.8
✓ Memory GPU: 10.00 GB

✓ Device aktif: cuda
✓ Semua fungsi dan metrics siap digunakan!

✓ Device aktif: cuda
✓ Semua fungsi dan metrics siap digunakan!


In [3]:
# Model definition from halfunet-training.ipynb
FILTER = 128

class LayerNormFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, weight, bias, eps):
        ctx.eps = eps
        N, C, H, W = x.size()
        mu = x.mean(1, keepdim=True)
        var = (x - mu).pow(2).mean(1, keepdim=True)
        y = (x - mu) / (var + eps).sqrt()
        ctx.save_for_backward(y, var, weight)
        y = weight.view(1, C, 1, 1) * y + bias.view(1, C, 1, 1)
        return y

    @staticmethod
    def backward(ctx, grad_output):
        eps = ctx.eps
        N, C, H, W = grad_output.size()
        y, var, weight = ctx.saved_tensors
        g = grad_output * weight.view(1, C, 1, 1)
        mean_g = g.mean(dim=1, keepdim=True)
        mean_gy = (g * y).mean(dim=1, keepdim=True)
        gx = 1. / torch.sqrt(var + eps) * (g - y * mean_gy - mean_g)
        return gx, (grad_output * y).sum(dim=3).sum(dim=2).sum(dim=0), grad_output.sum(dim=3).sum(dim=2).sum(
            dim=0), None

class LayerNorm2d(nn.Module):
    def __init__(self, channels, eps=1e-6):
        super(LayerNorm2d, self).__init__()
        self.register_parameter('weight', nn.Parameter(torch.ones(channels)))
        self.register_parameter('bias', nn.Parameter(torch.zeros(channels)))
        self.eps = eps

    def forward(self, x):
        return LayerNormFunction.apply(x, self.weight, self.bias, self.eps)

class SimpleGate(nn.Module):
    def forward(self, x):
        x1, x2 = x.chunk(2, dim=1)
        return x1 * x2

class NAFBlock(nn.Module):
    def __init__(self, c, DW_Expand=2, FFN_Expand=2, drop_out_rate=0.):
        super().__init__()
        dw_channel = c * DW_Expand
        self.conv1 = nn.Conv2d(in_channels=c, out_channels=dw_channel, kernel_size=1, padding=0, stride=1, groups=1,
                               bias=True)
        self.conv2 = nn.Conv2d(in_channels=dw_channel, out_channels=dw_channel, kernel_size=3, padding=1, stride=1,
                               groups=dw_channel, bias=True)
        self.conv3 = nn.Conv2d(in_channels=dw_channel // 2, out_channels=c, kernel_size=1, padding=0, stride=1,
                               groups=1, bias=True)
        self.sca = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels=dw_channel // 2, out_channels=dw_channel // 2, kernel_size=1, padding=0, stride=1,
                      groups=1, bias=True),
        )
        self.sg = SimpleGate()
        ffn_channel = FFN_Expand * c
        self.conv4 = nn.Conv2d(in_channels=c, out_channels=ffn_channel, kernel_size=1, padding=0, stride=1, groups=1,
                               bias=True)
        self.conv5 = nn.Conv2d(in_channels=ffn_channel // 2, out_channels=c, kernel_size=1, padding=0, stride=1,
                               groups=1, bias=True)
        self.norm1 = LayerNorm2d(c)
        self.norm2 = LayerNorm2d(c)
        self.dropout1 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.dropout2 = nn.Dropout(drop_out_rate) if drop_out_rate > 0. else nn.Identity()
        self.beta = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)
        self.gamma = nn.Parameter(torch.zeros((1, c, 1, 1)), requires_grad=True)

    def forward(self, inp):
        x = inp
        x = self.norm1(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.sg(x)
        x = x * self.sca(x)
        x = self.conv3(x)
        x = self.dropout1(x)
        y = inp + x * self.beta
        x = self.conv4(self.norm2(y))
        x = self.sg(x)
        x = self.conv5(x)
        x = self.dropout2(x)
        return y + x * self.gamma

class HalfUNet(nn.Module):
    def __init__(self, input_channels=3):
        super(HalfUNet, self).__init__()
        self.initial = nn.Conv2d(3, FILTER, 1, 1)
        self.conv1 = nn.Sequential(NAFBlock(FILTER), NAFBlock(FILTER))
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Sequential(NAFBlock(FILTER), NAFBlock(FILTER))
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Sequential(NAFBlock(FILTER), NAFBlock(FILTER))
        self.conv_up3 = nn.Conv2d(FILTER, FILTER * 16, 1, bias=False)
        self.up3 = nn.PixelShuffle(4)
        self.conv_up2 = nn.Conv2d(FILTER, FILTER * 4, 1, bias=False)
        self.up2 = nn.PixelShuffle(2)
        self.final_conv = nn.Conv2d(FILTER, 3, kernel_size=1)

    def forward(self, x):
        x = self.initial(x)
        x1 = self.conv1(x)
        pool1 = self.pool1(x1)
        x2 = self.conv2(pool1)
        pool2 = self.pool2(x2)
        x3 = self.conv3(pool2)
        up3 = self.conv_up3(x3)
        up3 = self.up3(up3)
        up2 = self.conv_up2(x2)
        up2 = self.up2(up2)
        up_scaled = x1 + up2 + up3
        output = self.final_conv(up_scaled)
        return output

In [4]:
class DenoisingDataset(Dataset):
    def __init__(self, noisy_images_path, gt_images_path, crop_size=256):
        self.noisy_paths = noisy_images_path
        self.gt_paths = gt_images_path
        self.crop_size = crop_size
        
        self.noisy_patches = []
        self.gt_patches = []

        # Pre-crop all images into patches during initialization
        for idx in tqdm(range(len(self.noisy_paths)), desc="Loading and Cropping Images"):
            noisy_img = cv2.cvtColor(cv2.imread(self.noisy_paths[idx]), cv2.COLOR_BGR2RGB)
            gt_img = cv2.cvtColor(cv2.imread(self.gt_paths[idx]), cv2.COLOR_BGR2RGB)
            
            h, w, _ = noisy_img.shape
            
            # Create overlapping patches for more data
            for i in range(0, h - self.crop_size + 1, self.crop_size // 2):
                for j in range(0, w - self.crop_size + 1, self.crop_size // 2):
                    noisy_patch = noisy_img[i:i+self.crop_size, j:j+self.crop_size]
                    gt_patch = gt_img[i:i+self.crop_size, j:j+self.crop_size]
                    
                    self.noisy_patches.append(noisy_patch)
                    self.gt_patches.append(gt_patch)

    def __len__(self):
        return len(self.noisy_patches)

    def __getitem__(self, idx):
        noisy_patch = self.noisy_patches[idx]
        gt_patch = self.gt_patches[idx]
        
        noisy_patch = torch.from_numpy(noisy_patch).permute(2, 0, 1).float() / 255.0
        gt_patch = torch.from_numpy(gt_patch).permute(2, 0, 1).float() / 255.0
        
        return noisy_patch, gt_patch

# Load data paths
DATA_PATH = 'Data/'
with open('Scene_Instances.txt') as f:
    instances = f.read().splitlines()

noisy_images_path = []
gt_images_path = []

for f in instances:
    if not f: continue
    p = os.path.join(DATA_PATH, f)
    try:
        # Find PNG files with GT and NOISY in their names
        all_files = os.listdir(p)
        gt_files = sorted([os.path.join(p, g) for g in all_files if 'GT' in g and g.endswith('.PNG')])
        noisy_files = sorted([os.path.join(p, g) for g in all_files if 'NOISY' in g and g.endswith('.PNG')])
        gt_images_path.extend(gt_files)
        noisy_images_path.extend(noisy_files)
    except OSError as e:
        print(f"Error accessing directory {p}: {e}")
        continue

print(f"✓ Found {len(noisy_images_path)} noisy images")
print(f"✓ Found {len(gt_images_path)} ground truth images")

denoising_dataset = DenoisingDataset(noisy_images_path=noisy_images_path, gt_images_path=gt_images_path)

# Split dataset
train_size = int(0.8 * len(denoising_dataset))
valid_size = int(0.1 * len(denoising_dataset))
test_size = len(denoising_dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(denoising_dataset, [train_size, valid_size, test_size])

# Create DataLoaders (num_workers=0 untuk Windows agar tidak freeze)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0, pin_memory=True)
print(f"\n✓ DataLoaders created (num_workers=0 untuk Windows stability)")

✓ Found 160 noisy images
✓ Found 160 ground truth images


Loading and Cropping Images:   0%|          | 0/160 [00:00<?, ?it/s]


✓ DataLoaders created (num_workers=0 untuk Windows stability)


In [5]:
def finetune_model(model, train_loader, valid_loader, epochs, device):
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs for fine-tuning.")
        model = torch.nn.DataParallel(model)
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # Reduced LR for fine-tuning
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    loss_func = torch.nn.L1Loss()

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        model.train()

        total_loss = 0.0
        total_psnr = 0.0
        total_ssim = 0.0

        loop = tqdm(enumerate(train_loader), total=len(train_loader))
        for batch_idx, (inputs, targets) in loop:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            psnr_val = psnr(outputs, targets)
            ssim_val = ssim(outputs, targets)

            total_loss += loss.item()
            total_psnr += psnr_val.item()
            total_ssim += ssim_val.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_description(f'Step [{batch_idx+1}/{len(train_loader)}]')
            loop.set_postfix(loss=loss.item(), psnr=psnr_val.item(), ssim=ssim_val.item())

        avg_loss = total_loss / len(train_loader)
        avg_psnr = total_psnr / len(train_loader)
        avg_ssim = total_ssim / len(train_loader)

        print(f"\nAvg Loss: {avg_loss:.4f}, Avg PSNR: {avg_psnr:.4f}, Avg SSIM: {avg_ssim:.4f}")

        avg_loss_val, avg_psnr_val, avg_ssim_val = evaluate_model(model, valid_loader, device)
        print(f"Val Loss: {avg_loss_val:.4f}, Val PSNR: {avg_psnr_val:.4f}, Val SSIM: {avg_ssim_val:.4f}")

        scheduler.step()

    # Remove DataParallel wrapper if present
    if isinstance(model, torch.nn.DataParallel):
        model = model.module
    return model

In [8]:
PRETRAINED_MODEL_PATH = '/kaggle/input/halfunet/pytorch/default/1/naf_128_double.pth'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Instantiate the correct model architecture
model = HalfUNet() 

# Load the state dictionary
pretrained_weights = torch.load(PRETRAINED_MODEL_PATH, map_location=device)

# The model was likely saved with DataParallel, so we handle the 'module.' prefix
if isinstance(pretrained_weights, dict) and any(k.startswith('module.') for k in pretrained_weights.keys()):
    new_state_dict = {k.replace('module.', ''): v for k, v in pretrained_weights.items()}
    model.load_state_dict(new_state_dict)
else:
    model.load_state_dict(pretrained_weights)

model.to(device)
print("Pre-trained model loaded successfully.")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/halfunet/pytorch/default/1/naf_128_double.pth'

In [6]:
# ============================================================================
# INISIALISASI MODEL
# ============================================================================

print("=" * 70)
print("INISIALISASI MODEL HALFUNET")
print("=" * 70)

# Buat model baru
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HalfUNet()
model = model.to(device)

print(f"\n✓ Model HalfUNet berhasil dibuat!")
print(f"✓ Device: {device}")
print(f"✓ Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"✓ Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Hitung ukuran model dalam MB
model_size = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 * 1024)
print(f"✓ Model size: {model_size:.2f} MB")

print("\n" + "=" * 70)
print("✓ MODEL SIAP UNTUK TRAINING!")
print("=" * 70)

INISIALISASI MODEL HALFUNET

✓ Model HalfUNet berhasil dibuat!
✓ Device: cuda
✓ Total parameters: 1,042,051
✓ Trainable parameters: 1,042,051
✓ Model size: 3.98 MB

✓ MODEL SIAP UNTUK TRAINING!

✓ Model HalfUNet berhasil dibuat!
✓ Device: cuda
✓ Total parameters: 1,042,051
✓ Trainable parameters: 1,042,051
✓ Model size: 3.98 MB

✓ MODEL SIAP UNTUK TRAINING!


In [7]:
# ============================================================================
# PINDAHKAN MODEL KE CPU (untuk menghindari lag saat training)
# ============================================================================

print("=" * 70)
print("MEMINDAHKAN MODEL KE CPU")
print("=" * 70)

# Pindahkan model dari GPU ke CPU
cpu_device = torch.device('cpu')
model = model.to(cpu_device)

print(f"\n✓ Model dipindahkan ke CPU")
print(f"✓ Device sekarang: {cpu_device}")
print(f"💡 Training akan lebih lambat di CPU, tapi tidak akan lag komputer")
print(f"💡 Estimasi waktu: ~5-7 menit untuk 50 patches × 3 epochs")

print("\n" + "=" * 70)
print("✓ MODEL SIAP UNTUK TRAINING DI CPU!")
print("=" * 70)

MEMINDAHKAN MODEL KE CPU

✓ Model dipindahkan ke CPU
✓ Device sekarang: cpu
💡 Training akan lebih lambat di CPU, tapi tidak akan lag komputer
💡 Estimasi waktu: ~5-7 menit untuk 50 patches × 3 epochs

✓ MODEL SIAP UNTUK TRAINING DI CPU!


In [13]:
# ============================================================================
# TRAINING MODEL DENGAN SAFEGUARDS
# ============================================================================

def train_model(model, train_loader, valid_loader, epochs=50, lr=1e-4, device=None, 
                patience=10, min_delta=1e-4):
    """
    Training function dengan monitoring lengkap dan early stopping
    
    Args:
        patience: Berapa epoch menunggu sebelum early stop jika tidak ada improvement
        min_delta: Minimum perubahan loss yang dianggap sebagai improvement
    """
    # Set default device jika tidak diberikan
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("=" * 70)
    print(f"MEMULAI TRAINING - {epochs} EPOCHS")
    print("=" * 70)
    
    model = model.to(device)
    
    # Setup optimizer dan scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )
    loss_func = torch.nn.L1Loss()
    
    # Untuk tracking best model
    best_val_loss = float('inf')
    best_model_weights = None
    best_epoch = 0
    
    # Early stopping
    patience_counter = 0
    
    # History untuk plotting
    train_losses = []
    val_losses = []
    train_psnrs = []
    val_psnrs = []
    
    print(f"\nKonfigurasi Training:")
    print(f"  - Learning Rate: {lr}")
    print(f"  - Max Epochs: {epochs}")
    print(f"  - Early Stopping Patience: {patience} epochs")
    print(f"  - Optimizer: Adam")
    print(f"  - Loss Function: L1Loss (MAE)")
    print(f"  - Device: {device}")
    
    for epoch in range(epochs):
        print(f"\n{'=' * 70}")
        print(f"EPOCH {epoch+1}/{epochs}")
        print('=' * 70)
        
        # ============ TRAINING PHASE ============
        model.train()
        total_loss = 0.0
        total_psnr = 0.0
        total_ssim = 0.0
        
        loop = tqdm(train_loader, desc=f"Training")
        for batch_idx, (inputs, targets) in enumerate(loop):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            
            # Check for NaN/Inf
            if torch.isnan(loss) or torch.isinf(loss):
                print("\n❌ ERROR: Loss is NaN/Inf! Training stopped.")
                print("Model terbaik tetap tersimpan.")
                return model, {
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'train_psnrs': train_psnrs,
                    'val_psnrs': val_psnrs,
                    'best_val_loss': best_val_loss,
                    'stopped_early': True,
                    'reason': 'NaN/Inf loss'
                }
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping untuk stabilitas
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            # Calculate metrics
            with torch.no_grad():
                psnr_val = psnr(outputs, targets)
                ssim_val = ssim(outputs, targets)
            
            total_loss += loss.item()
            total_psnr += psnr_val.item()
            total_ssim += ssim_val.item()
            
            # Update progress bar
            loop.set_postfix({
                'loss': f'{loss.item():.4f}',
                'psnr': f'{psnr_val.item():.2f}',
                'ssim': f'{ssim_val.item():.4f}'
            })
        
        # Calculate average training metrics
        avg_train_loss = total_loss / len(train_loader)
        avg_train_psnr = total_psnr / len(train_loader)
        avg_train_ssim = total_ssim / len(train_loader)
        
        train_losses.append(avg_train_loss)
        train_psnrs.append(avg_train_psnr)
        
        print(f"\n📊 Training Metrics:")
        print(f"   Loss: {avg_train_loss:.6f} | PSNR: {avg_train_psnr:.2f} dB | SSIM: {avg_train_ssim:.4f}")
        
        # ============ VALIDATION PHASE ============
        avg_val_loss, avg_val_psnr, avg_val_ssim = evaluate_model(model, valid_loader, device)
        
        val_losses.append(avg_val_loss)
        val_psnrs.append(avg_val_psnr)
        
        print(f"📊 Validation Metrics:")
        print(f"   Loss: {avg_val_loss:.6f} | PSNR: {avg_val_psnr:.2f} dB | SSIM: {avg_val_ssim:.4f}")
        
        # Calculate improvement
        improvement = best_val_loss - avg_val_loss
        
        # Update learning rate scheduler
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(avg_val_loss)
        new_lr = optimizer.param_groups[0]['lr']
        
        # Tampilkan jika learning rate berubah
        if old_lr != new_lr:
            print(f"📉 Learning Rate reduced: {old_lr:.6f} → {new_lr:.6f}")
        
        # Check for improvement
        if improvement > min_delta:
            best_val_loss = avg_val_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            best_epoch = epoch + 1
            patience_counter = 0
            
            print(f"✓ 🎉 NEW BEST MODEL! (Val Loss improved by {improvement:.6f})")
            
            # Save checkpoint (hanya best model, bukan setiap epoch)
            checkpoint_path = 'best_model.pth'
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': avg_train_loss,
                'val_loss': avg_val_loss,
                'val_psnr': avg_val_psnr,
                'val_ssim': avg_val_ssim,
            }, checkpoint_path)
            print(f"✓ Checkpoint saved: {checkpoint_path} (epoch {epoch+1})")
        else:
            patience_counter += 1
            print(f"⚠ No improvement for {patience_counter}/{patience} epochs")
            
            if patience_counter >= patience:
                print(f"\n🛑 EARLY STOPPING!")
                print(f"   Validation loss tidak membaik selama {patience} epochs")
                print(f"   Best model dari epoch {best_epoch} akan digunakan")
                break
        
        # Deteksi overfitting
        if len(train_losses) > 1:
            overfitting_ratio = (avg_train_loss / avg_val_loss) if avg_val_loss > 0 else 1
            if overfitting_ratio < 0.5:
                print(f"⚠️ WARNING: Possible overfitting detected!")
                print(f"   Train/Val loss ratio: {overfitting_ratio:.3f}")
                print(f"   Consider stopping training atau mengurangi complexity")
        
        # Monitor GPU memory (hanya jika training di GPU)
        if torch.cuda.is_available() and device.type == 'cuda':
            memory_allocated = torch.cuda.memory_allocated(device) / (1024**3)
            memory_reserved = torch.cuda.memory_reserved(device) / (1024**3)
            print(f"🔧 GPU Memory: Allocated={memory_allocated:.2f}GB, Reserved={memory_reserved:.2f}GB")
    
    print("\n" + "=" * 70)
    print("✓ TRAINING SELESAI!")
    print("=" * 70)
    
    # Load best model weights
    if best_model_weights is not None:
        model.load_state_dict(best_model_weights)
        print(f"✓ Best model loaded from epoch {best_epoch} (Val Loss: {best_val_loss:.6f})")
    else:
        print("⚠ No improvement was made. Keeping initial model.")
    
    # Plot training history
    plot_training_history(train_losses, val_losses, train_psnrs, val_psnrs)
    
    return model, {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_psnrs': train_psnrs,
        'val_psnrs': val_psnrs,
        'best_val_loss': best_val_loss,
        'best_epoch': best_epoch,
        'stopped_early': patience_counter >= patience
    }

def plot_training_history(train_losses, val_losses, train_psnrs, val_psnrs):
    """Plot training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot Loss
    ax1.plot(train_losses, label='Train Loss', marker='o', linewidth=2)
    ax1.plot(val_losses, label='Val Loss', marker='s', linewidth=2)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Tambahkan best point
    best_idx = val_losses.index(min(val_losses))
    ax1.plot(best_idx, val_losses[best_idx], 'r*', markersize=20, 
             label=f'Best (Epoch {best_idx+1})')
    ax1.legend(fontsize=11)
    
    # Plot PSNR
    ax2.plot(train_psnrs, label='Train PSNR', marker='o', linewidth=2)
    ax2.plot(val_psnrs, label='Val PSNR', marker='s', linewidth=2)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('PSNR (dB)', fontsize=12)
    ax2.set_title('Training and Validation PSNR', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3)
    
    # Tambahkan best point
    best_psnr_idx = val_psnrs.index(max(val_psnrs))
    ax2.plot(best_psnr_idx, val_psnrs[best_psnr_idx], 'r*', markersize=20,
             label=f'Best (Epoch {best_psnr_idx+1})')
    ax2.legend(fontsize=11)
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Training history saved: training_history.png")

# ============================================================================
# PILIH STRATEGI TRAINING
# ============================================================================
cpu_device = torch.device('cpu')

print("\n" + "=" * 70)
print("PILIH STRATEGI TRAINING:")
print("=" * 70)
print("\n1. STRATEGI AMAN (Recommended): 10 epoch dulu untuk test")
print("2. STRATEGI FULL: 50 epoch langsung")
print("3. STRATEGI CEPAT: Subset kecil data untuk quick test")
print("\nUncomment salah satu opsi di bawah:\n")

# OPSI 1: STRATEGI AMAN - Test 10 epoch dulu ✅ RECOMMENDED untuk training serius
# Estimasi waktu: 3-5 jam dengan RTX 3080 (tergantung jumlah data)
trained_model, history = train_model(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    epochs=10,        # Coba 10 epoch dulu
    lr=1e-4,
    patience=5,       # Early stop jika 5 epoch tidak ada improvement
    device=cpu_device  # PAKSA GUNAKAN CPU untuk menghindari lag
)

# OPSI 2: STRATEGI FULL - Langsung 50 epoch (jika sudah yakin)
# Estimasi waktu: 15-24 jam dengan RTX 3080 (untuk dataset penuh)
# HANYA untuk training final setelah hyperparameter tuning!
# trained_model, history = train_model(
#     model=model,
#     train_loader=train_loader,
#     valid_loader=valid_loader,
#     epochs=50,
#     lr=1e-4,
#     patience=10,
#     device=device
# )

# OPSI 3: STRATEGI CEPAT - Test dengan subset kecil (paling cepat)
# print("⏳ Membuat subset SANGAT kecil untuk quick test...")
# print(f"📊 Total train patches: {len(train_dataset)}")
# print(f"📊 Total valid patches: {len(valid_dataset)}")

# # Ambil hanya 50 patches untuk training CPU (lebih kecil karena CPU lebih lambat)
# quick_train_size = min(50, len(train_dataset))
# quick_valid_size = min(25, len(valid_dataset))

# small_train = Subset(train_dataset, range(0, quick_train_size))
# small_valid = Subset(valid_dataset, range(0, quick_valid_size))

# print(f"✓ Quick train size: {len(small_train)} patches")
# print(f"✓ Quick valid size: {len(small_valid)} patches")

# # Batch size lebih kecil untuk CPU (4 atau 8)
# small_train_loader = DataLoader(small_train, batch_size=4, shuffle=True, 
#                                 num_workers=0, pin_memory=False)  # pin_memory=False untuk CPU
# small_valid_loader = DataLoader(small_valid, batch_size=4, shuffle=False,
#                                 num_workers=0, pin_memory=False)

# print(f"✓ Train batches per epoch: {len(small_train_loader)}")
# print(f"✓ Valid batches per epoch: {len(small_valid_loader)}")

# # FORCE CPU MODE - Tidak peduli GPU tersedia atau tidak
# cpu_device = torch.device('cpu')
# print(f"\n🖥️  MENGGUNAKAN CPU MODE (untuk menghindari lag)")
# print(f"⏱️  Estimasi: ~5-7 menit untuk 3 epochs di CPU")
# print("✓ DataLoader siap, memulai training...\n")

# trained_model, history = train_model(
#     model=model,
#     train_loader=small_train_loader,
#     valid_loader=small_valid_loader,  # PENTING: Gunakan subset kecil untuk validasi!
#     epochs=3,  # Kurangi jadi 3 epoch saja untuk quick test
#     lr=1e-4,
#     patience=5,  # Tingkatkan patience (tidak akan early stop di 3 epoch)
#     device=cpu_device  # PAKSA GUNAKAN CPU!
# )


PILIH STRATEGI TRAINING:

1. STRATEGI AMAN (Recommended): 10 epoch dulu untuk test
2. STRATEGI FULL: 50 epoch langsung
3. STRATEGI CEPAT: Subset kecil data untuk quick test

Uncomment salah satu opsi di bawah:

MEMULAI TRAINING - 10 EPOCHS

Konfigurasi Training:
  - Learning Rate: 0.0001
  - Max Epochs: 10
  - Early Stopping Patience: 5 epochs
  - Optimizer: Adam
  - Loss Function: L1Loss (MAE)
  - Device: cpu

EPOCH 1/10

Konfigurasi Training:
  - Learning Rate: 0.0001
  - Max Epochs: 10
  - Early Stopping Patience: 5 epochs
  - Optimizer: Adam
  - Loss Function: L1Loss (MAE)
  - Device: cpu

EPOCH 1/10


Training:   0%|          | 0/11799 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 💾 LOAD CHECKPOINT (OPSIONAL)

Jika ingin melanjutkan training dari checkpoint yang tersimpan:

```python
# Load checkpoint untuk melanjutkan training
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ Model loaded from epoch {checkpoint['epoch']}")
print(f"✓ Val Loss: {checkpoint['val_loss']:.6f}")
print(f"✓ Val PSNR: {checkpoint['val_psnr']:.2f} dB")

# Lalu jalankan train_model() lagi dengan model yang sudah di-load
```

## ⚡ TIPS OPTIMASI KECEPATAN

**Saat ini menggunakan CPU MODE** (untuk menghindari lag komputer)

**Perbandingan Kecepatan:**
- **CPU Mode**: ~5-7 menit untuk 50 patches × 3 epochs (tidak lag komputer)
- **GPU Mode**: ~2-3 menit untuk 100 patches × 3 epochs (bisa lag komputer)

**Jika training CPU terlalu lambat:**

1. **Kurangi jumlah data training:**
   - Quick test CPU: 25-50 patches (5-7 menit)
   - Medium test CPU: 200-500 patches (30-60 menit)
   - Full training CPU: **TIDAK DISARANKAN** (bisa 48+ jam!)

2. **Batch size untuk CPU:**
   - CPU optimal: batch_size=2 atau 4 (jangan terlalu besar)
   - Lebih besar tidak selalu lebih cepat di CPU

3. **Kurangi jumlah epochs:**
   - Quick test: 3 epochs
   - Proper training: 5-10 epochs (untuk CPU)

4. **Jika ingin kembali ke GPU:**
   - Ubah `device=cpu_device` menjadi `device=device` di training cell
   - Tingkatkan `quick_train_size` ke 100-200
   - Tingkatkan `batch_size` ke 16-32

**File checkpoint yang disimpan:**
- `best_model.pth` - Model terbaik (diupdate setiap kali ada improvement)
- Tidak menyimpan per epoch untuk menghemat ruang disk

**💡 Rekomendasi:**
- CPU: Untuk quick test dan debugging (tidak lag)
- GPU: Untuk training serius (cepat tapi mungkin lag komputer)

## 🔄 SWITCH ANTARA CPU DAN GPU

**Jika ingin kembali menggunakan GPU** (setelah CPU mode):

```python
# Cek apakah GPU tersedia
if torch.cuda.is_available():
    gpu_device = torch.device('cuda')
    model = model.to(gpu_device)
    print(f"✓ Model dipindahkan ke GPU: {torch.cuda.get_device_name(0)}")
    
    # Lalu ubah parameter device di train_model():
    # device=gpu_device (bukan device=cpu_device)
else:
    print("❌ GPU tidak tersedia")
```

**Tips untuk mengurangi lag saat GPU mode:**
1. Tutup aplikasi lain (browser, game, dll)
2. Kurangi batch_size (16 → 8 atau 4)
3. Kurangi jumlah patches (100 → 50)
4. Monitor GPU usage di Task Manager

In [9]:
# ============================================================================
# EVALUASI DAN VISUALISASI HASIL
# ============================================================================

def visualize_results(model, test_loader, device, num_samples=5):
    """
    Visualisasi hasil denoising untuk memastikan model bekerja dengan baik
    """
    model.eval()
    model.to(device)
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    with torch.no_grad():
        for idx, (noisy, clean) in enumerate(test_loader):
            if idx >= num_samples:
                break
            
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            # Denoise
            denoised = model(noisy)
            
            # Ke CPU untuk visualisasi
            noisy_img = noisy[0].cpu().permute(1, 2, 0).numpy()
            clean_img = clean[0].cpu().permute(1, 2, 0).numpy()
            denoised_img = denoised[0].cpu().permute(1, 2, 0).numpy()
            
            # Calculate metrics untuk sample ini
            sample_psnr = psnr(denoised[0:1], clean[0:1]).item()
            sample_ssim = ssim(denoised[0:1], clean[0:1]).item()
            
            # Plot
            axes[idx, 0].imshow(np.clip(noisy_img, 0, 1))
            axes[idx, 0].set_title(f'Noisy Image {idx+1}', fontsize=12)
            axes[idx, 0].axis('off')
            
            axes[idx, 1].imshow(np.clip(denoised_img, 0, 1))
            axes[idx, 1].set_title(f'Denoised (PSNR: {sample_psnr:.2f} dB)', fontsize=12)
            axes[idx, 1].axis('off')
            
            axes[idx, 2].imshow(np.clip(clean_img, 0, 1))
            axes[idx, 2].set_title(f'Ground Truth (SSIM: {sample_ssim:.4f})', fontsize=12)
            axes[idx, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('denoising_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✓ Results saved: denoising_results.png")

# Uncomment untuk visualisasi hasil:
print("\n" + "=" * 70)
print("VISUALISASI HASIL DENOISING")
print("=" * 70)
visualize_results(trained_model, test_loader, device, num_samples=5)


VISUALISASI HASIL DENOISING


NameError: name 'trained_model' is not defined

## 🔬 PERBANDINGAN MODEL: best_model.pth vs optimized_halfunet_physical.pth

Mari kita evaluasi dan bandingkan kedua model untuk melihat mana yang lebih bagus!

In [14]:
# ============================================================================
# LOAD DAN EVALUASI KEDUA MODEL
# ============================================================================

import os

print("=" * 70)
print("PERBANDINGAN MODEL")
print("=" * 70)

# Device untuk evaluasi (gunakan CPU untuk konsistensi)
eval_device = torch.device('cpu')

# Cek apakah file model ada
best_model_path = 'best_model.pth'
optimized_model_path = 'optimized_halfunet_physical.pth'

print(f"\n📁 Mengecek file model...")
print(f"   best_model.pth: {'✓ Ada' if os.path.exists(best_model_path) else '✗ Tidak ada'}")
print(f"   optimized_halfunet_physical.pth: {'✓ Ada' if os.path.exists(optimized_model_path) else '✗ Tidak ada'}")

# Fungsi untuk load model
def load_model_for_comparison(model_path, device):
    """Load model checkpoint untuk evaluasi"""
    model = HalfUNet()
    
    if not os.path.exists(model_path):
        print(f"❌ File {model_path} tidak ditemukan!")
        return None, None
    
    try:
        checkpoint = torch.load(model_path, map_location=device)
        
        # Cek apakah checkpoint adalah dict dengan metadata atau langsung state_dict
        if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            metadata = {
                'epoch': checkpoint.get('epoch', 'N/A'),
                'val_loss': checkpoint.get('val_loss', 'N/A'),
                'val_psnr': checkpoint.get('val_psnr', 'N/A'),
                'val_ssim': checkpoint.get('val_ssim', 'N/A'),
            }
        else:
            # Jika langsung state_dict
            model.load_state_dict(checkpoint)
            metadata = None
        
        model = model.to(device)
        model.eval()
        return model, metadata
    except Exception as e:
        print(f"❌ Error loading {model_path}: {str(e)}")
        return None, None

print(f"\n🔄 Loading models...")

# Load best_model.pth (model hasil training baru)
model1, metadata1 = load_model_for_comparison(best_model_path, eval_device)
if model1 is not None:
    print(f"\n✓ Model 1 (best_model.pth) loaded successfully!")
    if metadata1:
        print(f"   Metadata:")
        print(f"   - Epoch: {metadata1['epoch']}")
        if metadata1['val_loss'] != 'N/A':
            print(f"   - Val Loss: {metadata1['val_loss']:.6f}")
            print(f"   - Val PSNR: {metadata1['val_psnr']:.2f} dB")
            print(f"   - Val SSIM: {metadata1['val_ssim']:.4f}")

# Load optimized_halfunet_physical.pth (model yang sudah ada)
model2, metadata2 = load_model_for_comparison(optimized_model_path, eval_device)
if model2 is not None:
    print(f"\n✓ Model 2 (optimized_halfunet_physical.pth) loaded successfully!")
    if metadata2:
        print(f"   Metadata:")
        print(f"   - Epoch: {metadata2['epoch']}")
        if metadata2['val_loss'] != 'N/A':
            print(f"   - Val Loss: {metadata2['val_loss']:.6f}")
            print(f"   - Val PSNR: {metadata2['val_psnr']:.2f} dB")
            print(f"   - Val SSIM: {metadata2['val_ssim']:.4f}")

print("\n" + "=" * 70)

PERBANDINGAN MODEL

📁 Mengecek file model...
   best_model.pth: ✓ Ada
   optimized_halfunet_physical.pth: ✓ Ada

🔄 Loading models...

✓ Model 1 (best_model.pth) loaded successfully!
   Metadata:
   - Epoch: 3
   - Val Loss: 0.044928
   - Val PSNR: 24.21 dB
   - Val SSIM: 0.5951

✓ Model 2 (optimized_halfunet_physical.pth) loaded successfully!



In [15]:
# ============================================================================
# EVALUASI KEDUA MODEL PADA TEST SET
# ============================================================================

print("=" * 70)
print("EVALUASI PADA TEST SET")
print("=" * 70)

# Pastikan metrics di CPU
ssim_cpu = StructuralSimilarityIndexMeasure(data_range=1.0).to(eval_device)
psnr_cpu = PeakSignalNoiseRatio(data_range=1.0).to(eval_device)

def evaluate_model_comparison(model, dataloader, device, model_name):
    """Evaluasi model dengan metrics lengkap"""
    if model is None:
        print(f"\n❌ {model_name} tidak dapat dievaluasi (model tidak loaded)")
        return None
    
    print(f"\n{'=' * 70}")
    print(f"EVALUATING: {model_name}")
    print('=' * 70)
    
    model.to(device)
    model.eval()
    
    total_loss = 0.0
    total_psnr = 0.0
    total_ssim = 0.0
    loss_func = torch.nn.L1Loss()
    
    print(f"⏳ Processing {len(dataloader)} batches...")
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(dataloader, desc=f"Evaluating {model_name}")):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            
            loss_val = loss_func(outputs, targets)
            psnr_val = psnr_cpu(outputs, targets)
            ssim_val = ssim_cpu(outputs, targets)
            
            total_loss += loss_val.item()
            total_psnr += psnr_val.item()
            total_ssim += ssim_val.item()
    
    avg_loss = total_loss / len(dataloader)
    avg_psnr = total_psnr / len(dataloader)
    avg_ssim = total_ssim / len(dataloader)
    
    print(f"\n📊 HASIL {model_name}:")
    print(f"   ├─ Loss (MAE):  {avg_loss:.6f}")
    print(f"   ├─ PSNR:        {avg_psnr:.2f} dB")
    print(f"   └─ SSIM:        {avg_ssim:.4f}")
    
    return {
        'loss': avg_loss,
        'psnr': avg_psnr,
        'ssim': avg_ssim
    }

# Evaluasi kedua model
results1 = evaluate_model_comparison(model1, test_loader, eval_device, "best_model.pth")
results2 = evaluate_model_comparison(model2, test_loader, eval_device, "optimized_halfunet_physical.pth")

print("\n" + "=" * 70)

EVALUASI PADA TEST SET

EVALUATING: best_model.pth
⏳ Processing 1475 batches...


Evaluating best_model.pth:   0%|          | 0/1475 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# ============================================================================
# PERBANDINGAN DAN VISUALISASI HASIL
# ============================================================================

print("=" * 70)
print("PERBANDINGAN HASIL")
print("=" * 70)

if results1 and results2:
    print("\n📊 RINGKASAN PERBANDINGAN:")
    print("=" * 70)
    
    # Header tabel
    print(f"{'Metric':<15} | {'best_model.pth':<20} | {'optimized_halfunet':<20} | {'Winner':<15}")
    print("-" * 70)
    
    # Loss (Lower is better)
    loss_winner = "best_model" if results1['loss'] < results2['loss'] else "optimized_halfunet"
    loss_diff = abs(results1['loss'] - results2['loss'])
    print(f"{'Loss (MAE)':<15} | {results1['loss']:<20.6f} | {results2['loss']:<20.6f} | {loss_winner:<15}")
    print(f"{'  (diff)':<15} | {'':<20} | {'':<20} | {loss_diff:.6f}")
    
    # PSNR (Higher is better)
    psnr_winner = "best_model" if results1['psnr'] > results2['psnr'] else "optimized_halfunet"
    psnr_diff = abs(results1['psnr'] - results2['psnr'])
    print(f"{'PSNR (dB)':<15} | {results1['psnr']:<20.2f} | {results2['psnr']:<20.2f} | {psnr_winner:<15}")
    print(f"{'  (diff)':<15} | {'':<20} | {'':<20} | {psnr_diff:.2f} dB")
    
    # SSIM (Higher is better)
    ssim_winner = "best_model" if results1['ssim'] > results2['ssim'] else "optimized_halfunet"
    ssim_diff = abs(results1['ssim'] - results2['ssim'])
    print(f"{'SSIM':<15} | {results1['ssim']:<20.4f} | {results2['ssim']:<20.4f} | {ssim_winner:<15}")
    print(f"{'  (diff)':<15} | {'':<20} | {'':<20} | {ssim_diff:.4f}")
    
    print("=" * 70)
    
    # Hitung overall winner
    score1 = 0
    score2 = 0
    
    if results1['loss'] < results2['loss']:
        score1 += 1
    else:
        score2 += 1
    
    if results1['psnr'] > results2['psnr']:
        score1 += 1
    else:
        score2 += 1
        
    if results1['ssim'] > results2['ssim']:
        score1 += 1
    else:
        score2 += 1
    
    print(f"\n🏆 OVERALL WINNER:")
    print("=" * 70)
    print(f"   best_model.pth:                 {score1}/3 metrics won")
    print(f"   optimized_halfunet_physical.pth: {score2}/3 metrics won")
    
    if score1 > score2:
        print(f"\n   ✅ best_model.pth LEBIH BAGUS!")
        print(f"      Model hasil training baru Anda menang di {score1} dari 3 metrics.")
    elif score2 > score1:
        print(f"\n   ✅ optimized_halfunet_physical.pth LEBIH BAGUS!")
        print(f"      Model yang sudah ada sebelumnya menang di {score2} dari 3 metrics.")
    else:
        print(f"\n   🤝 SERI! Kedua model memiliki performa yang sama.")
    
    print("=" * 70)
    
    # Analisis detail
    print(f"\n💡 ANALISIS:")
    
    if psnr_diff < 0.5:
        print(f"   • PSNR difference sangat kecil ({psnr_diff:.2f} dB) - kedua model hampir sama")
    elif psnr_diff < 1.0:
        print(f"   • PSNR difference kecil ({psnr_diff:.2f} dB) - perbedaan tidak terlalu signifikan")
    else:
        print(f"   • PSNR difference cukup besar ({psnr_diff:.2f} dB) - ada perbedaan yang jelas")
    
    if ssim_diff < 0.01:
        print(f"   • SSIM difference sangat kecil ({ssim_diff:.4f}) - kualitas visual hampir identik")
    elif ssim_diff < 0.05:
        print(f"   • SSIM difference kecil ({ssim_diff:.4f}) - kualitas visual sedikit berbeda")
    else:
        print(f"   • SSIM difference cukup besar ({ssim_diff:.4f}) - kualitas visual berbeda cukup jelas")
    
    print("\n" + "=" * 70)
    
elif results1 and not results2:
    print("\n⚠️  Hanya best_model.pth yang berhasil dievaluasi.")
    print("    optimized_halfunet_physical.pth tidak ditemukan atau error.")
elif results2 and not results1:
    print("\n⚠️  Hanya optimized_halfunet_physical.pth yang berhasil dievaluasi.")
    print("    best_model.pth tidak ditemukan atau error.")
else:
    print("\n❌ Tidak ada model yang berhasil dievaluasi.")
    print("   Pastikan file model ada di direktori yang benar.")

In [ ]:
# ============================================================================
# VISUALISASI SIDE-BY-SIDE COMPARISON
# ============================================================================

def visualize_model_comparison(model1, model2, test_loader, device, num_samples=3):
    """
    Visualisasi perbandingan hasil denoising dari kedua model
    """
    if model1 is None or model2 is None:
        print("❌ Tidak dapat visualisasi - salah satu model tidak loaded")
        return
    
    print("\n" + "=" * 70)
    print("VISUALISASI PERBANDINGAN")
    print("=" * 70)
    
    model1.eval()
    model2.eval()
    model1.to(device)
    model2.to(device)
    
    fig, axes = plt.subplots(num_samples, 5, figsize=(20, 4*num_samples))
    
    with torch.no_grad():
        for idx, (noisy, clean) in enumerate(test_loader):
            if idx >= num_samples:
                break
            
            noisy = noisy.to(device)
            clean = clean.to(device)
            
            # Denoise dengan kedua model
            denoised1 = model1(noisy)
            denoised2 = model2(noisy)
            
            # Ke CPU untuk visualisasi
            noisy_img = noisy[0].cpu().permute(1, 2, 0).numpy()
            clean_img = clean[0].cpu().permute(1, 2, 0).numpy()
            denoised1_img = denoised1[0].cpu().permute(1, 2, 0).numpy()
            denoised2_img = denoised2[0].cpu().permute(1, 2, 0).numpy()
            
            # Calculate metrics untuk sample ini
            psnr1 = psnr_cpu(denoised1[0:1], clean[0:1]).item()
            ssim1 = ssim_cpu(denoised1[0:1], clean[0:1]).item()
            psnr2 = psnr_cpu(denoised2[0:1], clean[0:1]).item()
            ssim2 = ssim_cpu(denoised2[0:1], clean[0:1]).item()
            
            # Plot
            # Column 1: Noisy
            axes[idx, 0].imshow(np.clip(noisy_img, 0, 1))
            axes[idx, 0].set_title(f'Sample {idx+1}\nNoisy Image', fontsize=11, fontweight='bold')
            axes[idx, 0].axis('off')
            
            # Column 2: Model 1 (best_model)
            axes[idx, 1].imshow(np.clip(denoised1_img, 0, 1))
            winner1 = "🏆 " if psnr1 > psnr2 else ""
            axes[idx, 1].set_title(f'{winner1}best_model.pth\nPSNR: {psnr1:.2f} dB\nSSIM: {ssim1:.4f}', 
                                   fontsize=10, color='green' if psnr1 > psnr2 else 'black')
            axes[idx, 1].axis('off')
            
            # Column 3: Model 2 (optimized_halfunet)
            axes[idx, 2].imshow(np.clip(denoised2_img, 0, 1))
            winner2 = "🏆 " if psnr2 > psnr1 else ""
            axes[idx, 2].set_title(f'{winner2}optimized_halfunet\nPSNR: {psnr2:.2f} dB\nSSIM: {ssim2:.4f}', 
                                   fontsize=10, color='green' if psnr2 > psnr1 else 'black')
            axes[idx, 2].axis('off')
            
            # Column 4: Ground Truth
            axes[idx, 3].imshow(np.clip(clean_img, 0, 1))
            axes[idx, 3].set_title(f'Ground Truth\n(Target)', fontsize=11, fontweight='bold')
            axes[idx, 3].axis('off')
            
            # Column 5: Difference heatmap (antara best_model dan optimized)
            diff = np.abs(denoised1_img - denoised2_img)
            diff_magnitude = np.mean(diff, axis=2)  # Average across RGB
            im = axes[idx, 4].imshow(diff_magnitude, cmap='hot', vmin=0, vmax=0.1)
            axes[idx, 4].set_title(f'Difference\nMean: {np.mean(diff):.4f}', fontsize=10)
            axes[idx, 4].axis('off')
            plt.colorbar(im, ax=axes[idx, 4], fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n✓ Comparison visualization saved: model_comparison.png")
    print("=" * 70)

# Jalankan visualisasi
if model1 and model2:
    visualize_model_comparison(model1, model2, test_loader, eval_device, num_samples=3)
else:
    print("\n⚠️  Tidak dapat membuat visualisasi - salah satu model tidak loaded")